# Exploración de un modelo entrenado con Word2Vec para el idioma español

En esta sección, exploraremos las cualidades de un modelo entrenado con el algoritmo Word2Vec para el idioma español. Este notebook requiere tener instaladas las siguientes librerias:
- gensim
- numpy
- tensorflow (opcional)

In [ ]:
import sys, os, io, pathlib
import numpy as np

from gensim.models import KeyedVectors
from gensim.test.utils import datapath

## Cargamos un modelo pre-entrenado en Español

Utilizaremos un modelo pre-entrenado de Word2Vec que fué entrenado con un corpus en español (Spanish CoNLL17 corpus). Este modelo se entrenó trabajando con una ventana de 10 palabras, con el algoritmo Continuous bag of words y un negative-sampling de 5 palabras. Este modelo está almacenado en formato binario, por lo que lo cargamos utilizando el objeto KeyedVectors y el metodo load_word2vec_format

Descargamos el modelo

In [ ]:
!mkdir -p ./Models/Word2Vec
!wget https://santiagxf.blob.core.windows.net/public/Word2Vec/model.bin --directory-prefix ./Models/Word2Vec

Cargamos el embedding

In [ ]:
path = os.path.abspath('Models/Word2Vec/model.bin')
embeddings = KeyedVectors.load_word2vec_format(datapath(path), binary=True)

Podemos objetener algunas propiedades del modelo

In [ ]:
print ("El tamaño del vocabulario es:", len(embeddings.vocab))
print("El tamaño de los vectores es:", embeddings.vector_size)

## Exploramos las representaciones
Podemos comenzar a indagar como lucen las representaciones vectoriales que se aprendieron utilizando el método most_similar. Este método nos devuelve las 10 primeras palabras más similares a la palabra que indicamos. La similaridad se computa utilizando cosine-similarity de las representaciones vectoriales de cada una de las palabras.

In [ ]:
embeddings.most_similar("reina")

También podemos consultar cuales son las palabras más disimilares a una determinada palabra. Esto lo hacemos especificando el parametro negative dentro del método most_similar

In [ ]:
embeddings.most_similar(negative="reina")

## Analogias

El método anterior no es muy util en general, sin embargo nos permite introducir el concepto de aritmética sobre estos mismos vectores. Por ejemplo: Pordiamos consultar cual es la analogia de dos palabras. El siguiente ejemplo se leería así: A lo que Paris es a Francia, ¿cúal es la analogia de Madrid?. Esto se resolveria tomando "Francia", quitandole "Paris" y agregandole "Madrid"

<img src="Docs/word2vec-math.png" width="500" />

In [ ]:
embeddings.most_similar(positive=["francia", "madrid"], negative=["paris"])[0][0]

Estos nos dice que quizás este espacio vectorial no es solamente un espacio donde vectores que están cerca los unos de los otros en el espacio tienen significados similares, sino que en realidad capturan el significado en una forma más produnda. Concretamente, que hay "direcciones de significados" en el espacio donde uno de puede mover. Veamos algunos ejemplos:

En la dirección de "hacedor"
 - Conducir es a conductor lo que limpiar es a?

In [ ]:
embeddings.most_similar(positive=[], negative=[])[0][0]

En la dirección "bebidas"
 - El vino es a Francia lo que el whisky es a?

In [ ]:
embeddings.most_similar(positive=[], negative=[])[0][0]

En la dirección "extremar"
 - Bueno es a genial lo que malo es a?

In [ ]:
embeddings.most_similar(positive=[], negative=[])[0][0]

## Visualización del espacio contino de word2vec

El siguiente código utiliza un plugin de Tensorflow llamado Tensorboard que permite hacer una proyección en un espacio 3D (para luego hacer una proyección en un espacio 2D - el de la pantalla) de las representaciones de cada una de las palabras que están en el vocabulario. Esta visualización requiere tener instalado TensorFlow 2.0

In [ ]:
import tensorflow as tf
from tqdm import tqdm
from tensorboard.plugins import projector

In [ ]:
logdir = '/tmp/logdir'

In [ ]:
!mkdir -p '/tmp/logdir'

In [ ]:
weights = tf.Variable(embeddings.vectors)
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(logdir, 'embedding.ckpt'))

El siguiente codigo genera los "labels" asociados al espacio vectorial que cargamos anteriormente. Pueden descargar estos labels ya preprocesados desde:

In [ ]:
!wget https://santiagxf.blob.core.windows.net/public/Word2Vec/meta.tsv --directory-prefix '/tmp/logdir'

In [ ]:
with io.open(logdir + '/meta.tsv', 'w', encoding='utf-8') as metadata_file:
    with io.open(logdir + '/vecs.tsv', 'w', encoding='utf-8') as vectors_file:
        for index in tqdm(range(len(embeddings.index2word))):
            word = embeddings.index2word[index]
            vec = embeddings.vectors[index]
            metadata_file.write(word + "\n")
            vectors_file.write('\t'.join([str(x) for x in vec]) + "\n")

In [ ]:
# Set up config
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'meta.tsv'
projector.visualize_embeddings(logdir, config)

Cargamos la extensión de Google Colab Tensorboard

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logdir

Nota: Actualmente hay un problema con Google Colab con la carga de Projector en Tensorboard. Al parecer es debido al modulo tfjs. Si quieren probar esta funcionalidad tendrán que ejecutar este codigo en su equipo local. Para hacer esto deberan descargar este notebook en su equipo local y ejecutar finalmente:

!pip install tensorboard
!tensorboard --logdir ./logdir